# Data Class

In [1]:
import random


class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"


class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:  #score of 4 or 5
            return Sentiment.POSITIVE


class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    def evenly_distribution(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))

        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)



# Load Data 

In [ ]:
import json

file_name = r'ignore/dataBooks_5_new.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        #         print(review['reviewText'])
        #         print(review['overall'])
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text


# Prep Data

In [ ]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)


In [ ]:
train_container.evenly_distribution()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribution()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

## Bag of words vectorization 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())



# Classification

### Linear SVM

In [ ]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])


### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

# Evaluation

In [ ]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

In [ ]:
# F1 scores
from sklearn.metrics import f1_score

# f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])



In [ ]:
test_set = ['not great', 'i dont like it', 'relax', 'trash', 'absolute junk']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

## Tuning our model (with grid search)

In [ ]:
# from sklearn.model_selection import GridSearchCV
#
# parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)}
#
# svc = svm.SVC()
# clf = GridSearchCV(svc, parameters, cv=5)
# clf.fit(train_x_vectors, train_y)




In [ ]:
# print(clf.score(test_x_vectors, test_y))

## Saving Model

In [ ]:
import pickle

with open(r'./models/sentiment_classifier.pkl', 'wb') as f:
    pickle.dump(clf_svm, f)

with open(r'./models/vectorizer.pkl', 'wb') as g:
    pickle.dump(vectorizer, g)

### load model

In [ ]:
import pickle

with open(r'./models/sentiment_classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

with open(r'./models/vectorizer.pkl', 'rb') as g:
    loaded_vectorizer = pickle.load(g)

In [ ]:
# print(test_x[56])
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

t1 = ['there was nothing interesting in the book, waste of time', 'absolute disgusting', 'loved it', 'amazing content', 'not a good one']
test_case = loaded_vectorizer.transform(t1)
loaded_clf.predict(test_case)